In [1]:
# ============================================================================
# ANALYTICS Y MACHINE LEARNING CON SPARKML
# ============================================================================
# Objetivos de aprendizaje:
# 1. Comprender arquitectura de SparkML y Pipelines
# 2. Transformar datos para ML
# 3. Construir y evaluar modelos de clasificación en Spark
# 4. Comparar SparkML con scikit-learn en escalabilidad
# Dataset: NYC Taxi (propina alta sí/no)
# ============================================================================

print("""
╔════════════════════════════════════════════════════════════════════════╗
║                                          ║
║            ANALYTICS Y MACHINE LEARNING CON SPARKML                 ║
╚════════════════════════════════════════════════════════════════════════╝

En esta semana aprenderás a:

 ✓ Usar SparkML Pipeline API para ML reproducible
 ✓ Transformar features con VectorAssembler y otros transformers
 ✓ Entrenar clasificador masivo (ej. Regresión Logística)
 ✓ Evaluar modelos (accuracy, ROC-AUC, F1)
 ✓ Optimizar hiperparámetros de forma distribuida

💡 SparkML está diseñado para trabajar con datasets masivos y DataFrames.
🎯 Caso práctico: ¿Viaje tendrá propina alta?
""")



╔════════════════════════════════════════════════════════════════════════╗
║                                          ║
║            ANALYTICS Y MACHINE LEARNING CON SPARKML                 ║
╚════════════════════════════════════════════════════════════════════════╝

En esta semana aprenderás a:

 ✓ Usar SparkML Pipeline API para ML reproducible
 ✓ Transformar features con VectorAssembler y otros transformers
 ✓ Entrenar clasificador masivo (ej. Regresión Logística)
 ✓ Evaluar modelos (accuracy, ROC-AUC, F1)
 ✓ Optimizar hiperparámetros de forma distribuida

💡 SparkML está diseñado para trabajar con datasets masivos y DataFrames.
🎯 Caso práctico: ¿Viaje tendrá propina alta?



In [2]:
# Instalación rápida
!apt-get update -qq
!apt-get install -y openjdk-11-jdk-headless -qq
!pip install -q pyspark findspark

# Configurar
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Inicializar
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MiApp") \
    .getOrCreate()

print("✅ Spark listo!")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package openjdk-11-jre-headless:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../openjdk-11-jre-headless_11.0.29+7-1ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-11-jre-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
Selecting previously unselected package openjdk-11-jdk-headless:amd64.
Preparing to unpack .../openjdk-11-jdk-headless_11.0.29+7-1ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-11-jdk-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
Setting up openjdk-11-jre-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jjs to provide /usr/bin/jjs (jjs) in auto mode
update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/rmid to provide /usr/bin/rmid

In [3]:
from pyspark.sql.functions import col, when, hour, dayofweek, avg, count, round as spark_round
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, DoubleType, StringType

# Importar SparkML
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [4]:
# ============================================================================
# GENERACIÓN DEL DATASET NYC TAXI PARA MACHINE LEARNING
# ============================================================================

print("\n" + "="*70)
print("📂 GENERACIÓN DE DATOS PARA MACHINE LEARNING")
print("="*70)

print("""
💡 Importante:
  Usaremos datos sintéticos que simulan viajes de taxi.
  - Objetivo (target): high_tip (1 si tip/fare > 20%)
  - Features: trip_distance, passenger_count, hour_of_day, day_of_week, trip_duration
""")

def generar_datos_taxi_ml(n_registros=15000):
    import random
    from datetime import datetime, timedelta
    random.seed(42)
    base_date = datetime(2024, 1, 1)
    data = []
    for i in range(n_registros):
        trip_id = i + 1
        pickup_dt = base_date + timedelta(
            days=random.randint(0, 30),
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59)
        )
        trip_duration_minutes = random.randint(5, 90)
        dropoff_dt = pickup_dt + timedelta(minutes=trip_duration_minutes)
        passenger_count = random.randint(1, 6)
        trip_distance = round(random.uniform(0.5, 25.0), 2)
        fare_amount = round(2.5 + (trip_distance * 2.5) + random.uniform(-3, 5), 2)
        fare_amount = max(fare_amount, 3.0)  # Mínimo $3
        hour_of_day = pickup_dt.hour
        tip_factor = 0.15
        if hour_of_day in [7, 8, 17, 18, 19, 20]: tip_factor += 0.08
        if trip_distance > 10: tip_factor += 0.05
        if passenger_count >= 3: tip_factor += 0.04
        tip_factor += random.uniform(-0.05, 0.10)
        tip_factor = max(0, min(tip_factor, 0.40))
        tip_amount = round(fare_amount * tip_factor, 2)
        high_tip = 1 if fare_amount > 0 and (tip_amount/fare_amount) > 0.2 else 0
        data.append((trip_id, pickup_dt, dropoff_dt, passenger_count, trip_distance, fare_amount, tip_amount, high_tip, trip_duration_minutes))
    return data

schema = StructType([
    StructField("trip_id", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("high_tip", IntegerType(), True),
    StructField("trip_duration", IntegerType(), True),
])

print("⏳ Generando 15,000 registros de viajes de taxi...")
datos_taxi = generar_datos_taxi_ml(15000)
df_taxi = spark.createDataFrame(datos_taxi, schema=schema)
print(f"✅ Datos generados. Total de registros: {df_taxi.count()}")
df_taxi.show(10, truncate=False)
print("\n🎯 Distribución de la variable objetivo (high_tip):")
df_taxi.groupBy("high_tip").count().show()



📂 GENERACIÓN DE DATOS PARA MACHINE LEARNING

💡 Importante:
  Usaremos datos sintéticos que simulan viajes de taxi.
  - Objetivo (target): high_tip (1 si tip/fare > 20%)
  - Features: trip_distance, passenger_count, hour_of_day, day_of_week, trip_duration

⏳ Generando 15,000 registros de viajes de taxi...
✅ Datos generados. Total de registros: 15000
+-------+--------------------+---------------------+---------------+-------------+-----------+----------+--------+-------------+
|trip_id|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|fare_amount|tip_amount|high_tip|trip_duration|
+-------+--------------------+---------------------+---------------+-------------+-----------+----------+--------+-------------+
|1      |2024-01-21 03:01:00 |2024-01-21 03:41:00  |2              |5.97         |20.32      |4.09      |1       |40           |
|2      |2024-01-29 17:05:00 |2024-01-29 18:25:00  |4              |1.28         |3.45       |0.88      |1       |80           |
|3 

In [5]:
# ============================================================================
# PREPARACIÓN DE FEATURES PARA MACHINE LEARNING
# ============================================================================

print("="*70)
print("🔬 TRANSFORMACIÓN DE VARIABLES PARA ML CON SPARK")
print("="*70)

print("""
Las variables seleccionadas para el modelo serán:
    - trip_distance
    - passenger_count
    - trip_duration
    - hour_of_day (extraída de tpep_pickup_datetime)
    - day_of_week (extraída de tpep_pickup_datetime)
""")

# Extraer variables temporales
from pyspark.sql.functions import hour, dayofweek

df_ml = df_taxi.withColumn("hour_of_day", hour(col("tpep_pickup_datetime"))) \
    .withColumn("day_of_week", dayofweek(col("tpep_pickup_datetime")))

features = [
    "trip_distance",
    "passenger_count",
    "trip_duration",
    "hour_of_day",
    "day_of_week"
]

target = "high_tip"

print(f"✅ Variables para VectorAssembler: {features}")


🔬 TRANSFORMACIÓN DE VARIABLES PARA ML CON SPARK

Las variables seleccionadas para el modelo serán:
    - trip_distance
    - passenger_count
    - trip_duration
    - hour_of_day (extraída de tpep_pickup_datetime)
    - day_of_week (extraída de tpep_pickup_datetime)

✅ Variables para VectorAssembler: ['trip_distance', 'passenger_count', 'trip_duration', 'hour_of_day', 'day_of_week']


In [6]:
# ============================================================================
# PIPELINE DE MACHINE LEARNING CON SPARKML
# ============================================================================

print("="*70)
print("⚡ PIPELINE: PREPARAR, ENTRENAR Y EVALUAR UN MODELO EN SPARKML")
print("="*70)

from pyspark.ml.feature import VectorAssembler, StandardScaler

# Paso 1: Ensamblar variables numéricas
assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Paso 2: Escalado de features
scaler = StandardScaler(inputCol="features_vector", outputCol="features", withMean=True, withStd=True)

# Paso 3: Modelo de clasificación: Regresión Logística
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol=target)

# Construir pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, scaler, lr])

print("✅ Pipeline construido: [VectorAssembler → StandardScaler → LogisticRegression]")


⚡ PIPELINE: PREPARAR, ENTRENAR Y EVALUAR UN MODELO EN SPARKML
✅ Pipeline construido: [VectorAssembler → StandardScaler → LogisticRegression]


In [8]:
# ============================================================================
# ENTRENAMIENTO Y EVALUACIÓN DE UN MODELO DE CLASIFICACIÓN BINARIA
# ============================================================================

print("="*70)
print("🚀 ENTRENANDO Y EVALUANDO REGRESIÓN LOGÍSTICA EN SPARKML")
print("="*70)

# Dividir en training/test
train_df, test_df = df_ml.randomSplit([0.7, 0.3], seed=42)
print(f"Registros para entrenamiento: {train_df.count()}, para testing: {test_df.count()}")

# Ajustar pipeline al set de entrenamiento
modelo = pipeline.fit(train_df)

# Predicción sobre test
predicciones = modelo.transform(test_df)

print("\n🎯 Ejemplo de predicciones:")
predicciones.select("features", "high_tip", "prediction", "probability").show(5)

# Evaluación
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol=target, rawPredictionCol="prediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predicciones)

accuracy = predicciones.filter(col("high_tip") == col("prediction")).count() / test_df.count() * 100

print(f"\n✅ ROC-AUC: {roc_auc:.2f}")
print(f"✅ ACCURACY: {accuracy:.2f}%")


🚀 ENTRENANDO Y EVALUANDO REGRESIÓN LOGÍSTICA EN SPARKML
Registros para entrenamiento: 10649, para testing: 4351

🎯 Ejemplo de predicciones:
+--------------------+--------+----------+--------------------+
|            features|high_tip|prediction|         probability|
+--------------------+--------+----------+--------------------+
|[0.52991045230484...|       1|       1.0|[0.09008428361714...|
|[-1.0604821066281...|       1|       1.0|[0.19388431991966...|
|[-0.8008551322438...|       1|       1.0|[0.16049387485182...|
|[-0.8774663705867...|       1|       1.0|[0.12666765411161...|
|[0.22204677229730...|       1|       1.0|[0.08915727533591...|
+--------------------+--------+----------+--------------------+
only showing top 5 rows


✅ ROC-AUC: 0.56
✅ ACCURACY: 78.60%


In [ ]:
# ============================================================================
# UPDATE:  Usar one-hot encoding para las variables hour_of_day y day_of_week
# ============================================================================

# Extraer variables temporales
from pyspark.sql.functions import hour, dayofweek, col
df_ml = df_taxi.withColumn("hour_of_day", hour(col("tpep_pickup_datetime"))) \
    .withColumn("day_of_week", dayofweek(col("tpep_pickup_datetime")))

print("="*70)
print("⚡ PIPELINE: PREPARAR, ENTRENAR Y EVALUAR UN MODELO EN SPARKML")
print("="*70)

from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, StringIndexer

# Paso 1: Convertir variables temporales a índices y luego a one-hot encoding
hour_indexer = StringIndexer(inputCol="hour_of_day", outputCol="hour_indexed")
hour_encoder = OneHotEncoder(inputCol="hour_indexed", outputCol="hour_encoded")

day_indexer = StringIndexer(inputCol="day_of_week", outputCol="day_indexed")
day_encoder = OneHotEncoder(inputCol="day_indexed", outputCol="day_encoded")

# Paso 2a: Ensamblar SOLO las variables numéricas para escalar
numerical_features = [
    "trip_distance",
    "passenger_count",
    "trip_duration"
]

numerical_assembler = VectorAssembler(inputCols=numerical_features, outputCol="numerical_features")

# Paso 2b: Escalar SOLO las variables numéricas
scaler = StandardScaler(inputCol="numerical_features", outputCol="numerical_features_scaled",
                        withMean=True, withStd=True)

# Paso 3: Ensamblar TODO (numéricas escaladas + one-hot sin escalar)
final_features = [
    "numerical_features_scaled",
    "hour_encoded",
    "day_encoded"
]

final_assembler = VectorAssembler(inputCols=final_features, outputCol="features")

print(f"✅ Variables numéricas a escalar: {numerical_features}")
print(f"✅ Features finales ensambladas: {final_features}")

# Paso 4: Modelo de clasificación: Regresión Logística
from pyspark.ml.classification import LogisticRegression
target = "high_tip"
lr = LogisticRegression(featuresCol="features", labelCol=target)

# Construir pipeline completo
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[
    hour_indexer, hour_encoder,
    day_indexer, day_encoder,
    numerical_assembler, scaler,
    final_assembler,
    lr
])

print("✅ Pipeline construido: [StringIndexer → OneHotEncoder → VectorAssembler(numéricos) → StandardScaler → VectorAssembler(final) → LogisticRegression]")
print("="*70)
print("🚀 ENTRENANDO Y EVALUANDO REGRESIÓN LOGÍSTICA EN SPARKML")
print("="*70)

# Dividir en training/test
train_df, test_df = df_ml.randomSplit([0.7, 0.3], seed=42)
print(f"Registros para entrenamiento: {train_df.count()}, para testing: {test_df.count()}")

# Ajustar pipeline al set de entrenamiento
modelo = pipeline.fit(train_df)

# Predicción sobre test
predicciones = modelo.transform(test_df)

print("\n🎯 Ejemplo de predicciones:")
predicciones.select("features", "high_tip", "prediction", "probability").show(5)

# Evaluación
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol=target, rawPredictionCol="prediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predicciones)

accuracy = predicciones.filter(col("high_tip") == col("prediction")).count() / test_df.count() * 100

print(f"\n📊 RESULTADOS DEL MODELO:")
print(f"   - ROC-AUC: {roc_auc:.4f}")
print(f"   - Accuracy: {accuracy:.2f}%")
print("="*70)

⚡ PIPELINE: PREPARAR, ENTRENAR Y EVALUAR UN MODELO EN SPARKML
✅ Variables numéricas a escalar: ['trip_distance', 'passenger_count', 'trip_duration']
✅ Features finales ensambladas: ['numerical_features_scaled', 'hour_encoded', 'day_encoded']
✅ Pipeline construido: [StringIndexer → OneHotEncoder → VectorAssembler(numéricos) → StandardScaler → VectorAssembler(final) → LogisticRegression]
🚀 ENTRENANDO Y EVALUANDO REGRESIÓN LOGÍSTICA EN SPARKML
Registros para entrenamiento: 10649, para testing: 4351

🎯 Ejemplo de predicciones:
+--------------------+--------+----------+--------------------+
|            features|high_tip|prediction|         probability|
+--------------------+--------+----------+--------------------+
|(32,[0,1,2,25],[0...|       1|       1.0|[0.08898505517088...|
|(32,[0,1,2,5,26],...|       1|       1.0|[0.01388994549747...|
|(32,[0,1,2,8],[-0...|       1|       1.0|[0.24674475571671...|
|(32,[0,1,2,5],[-0...|       1|       1.0|[0.01031594027385...|
|(32,[0,1,2,3,31],...| 

In [ ]:
# Tarea

#1. Modifica el pipeline para usar RandomForestClassifier y compara métricas.
#2. Cambia la variable objetivo a propina mayor a 10% (tip_amount/fare_amount > 0.1) y repite el proceso.